## 참조 자료


https://zhuanlan.zhihu.com/p/72622208

In [1]:
import tensorflow as tf

##  일반 파이썬 함수에 데코레이터 처리

In [3]:
@tf.function
def add(a, b):
    return a+b

In [4]:
add(tf.ones([2,2]), tf.ones([2,2]))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

## 미분 처리 

In [5]:
v = tf.Variable(2.0)

In [6]:
with tf.GradientTape() as tape:
    res = add(v, 1.0)



In [7]:
tape.gradient(res, v)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

## 레이어 처리 

In [8]:
@tf.function
def dense_layer(x, w, b):
    return add(tf.matmul(x, w), b)


In [9]:
dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

## 함수에서 처리되는 결과 확인 

In [16]:
@tf.function
def double(a):
    print('인자 ：',a)
    return a + a

In [17]:
print('결과 1 :',double(tf.constant(1)))

인자 ： Tensor("a:0", shape=(), dtype=int32)
결과 1 : tf.Tensor(2, shape=(), dtype=int32)


In [18]:
print('결과 2 :',double(tf.constant(1.1)))

인자 ： Tensor("a:0", shape=(), dtype=float32)
결과 2 : tf.Tensor(2.2, shape=(), dtype=float32)


In [19]:
print('결과 3:',double(tf.constant('c')))

인자 ： Tensor("a:0", shape=(), dtype=string)
결과 3: tf.Tensor(b'cc', shape=(), dtype=string)


## 특정 매개변수 지정하기

In [27]:
double_strings = double.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.string))

In [28]:
print("실행 추적 기능")
print(double_strings(tf.constant("a")))
print(double_strings(a=tf.constant("b")))


실행 추적 기능
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)


## 예러 확인하기

In [29]:
double_strings(tf.constant(100))

InvalidArgumentError: cannot compute __inference_double_131 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_131]

In [30]:
import contextlib

# 构建包含上下文管理器的函数，使其可以在with中使用
@contextlib.contextmanager
def assert_raises(error_class):
    try:
        yield
    except error_class as e:
        print('Caught expected exception \n  {}: {}'.format(error_class, e))
    except Exception as e:
        print('Got unexpected exception \n  {}: {}'.format(type(e), e))
    else:
        raise Exception('Expected {} to be raised but no error was raised!'.format(
            error_class))

In [31]:
with assert_raises(tf.errors.InvalidArgumentError):
    double_strings(tf.constant(1))

Caught expected exception 
  <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>: cannot compute __inference_double_131 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_131]


## 함수 내의 매개변수 생성 방식

In [37]:
def train_one_step():
    pass

@tf.function
def train(num_steps):
    print("追踪： num_steps = {}".format(num_steps))
    for _ in tf.range(num_steps):
        train_one_step()


## 파이썬은 매개변수를 함수 호출할 때마다 생성한다 

In [38]:
train(num_steps=10)
train(num_steps=20)


追踪： num_steps = 10
追踪： num_steps = 20


## 텐서플로우는 함수를 호출할 때 하나의 매개변수만 만든다 

In [39]:
# 使用tensor，同类型不会重复追踪
train(num_steps=tf.constant(10))
train(num_steps=tf.constant(20))


追踪： num_steps = Tensor("num_steps:0", shape=(), dtype=int32)


## 인자의 매개변수가 변경되면 같이 변경된다

In [40]:

# 使用tensor，类型不同才会有新的追踪，（前一个单元格已追踪int型，所以该处不追踪）
train(num_steps=tf.constant(10, dtype=tf.int32))
train(num_steps=tf.constant(20.6))


追踪： num_steps = Tensor("num_steps:0", shape=(), dtype=float32)


## 동일한 값을 전달하면  기존에 있는 값을 처리한다

      함수를 처리할 때 내부의 매개변수를 유지..
      

In [50]:
@tf.function
def f(x):
    print("입력 ：", x)
    tf.print('호출 ：', x)


In [51]:
f(1)
f(1)

입력 ： 1
호출 ： 1
호출 ： 1


In [52]:
f(2)

입력 ： 2
호출 ： 2


## 매번 실행할 때마다 새로운 입력과 아웃을 처리

      함수, 입력과 아웃을 지정
      단점은 분산처리할 때 이슈가 있음

In [53]:
external_list = []

def side_effect(x):
    print('Python side effect')
    external_list.append(x)

@tf.function
def f(x):
    tf.py_function(side_effect, inp=[x], Tout=[])


In [54]:
f(1)
f(1)
f(1)
print(external_list)

Python side effect
Python side effect
Python side effect
[<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=1>]


In [55]:
external_var = tf.Variable(0)
@tf.function
def buggy_consume_next(iterator):
    external_var.assign_add(next(iterator))
    tf.print('external_var:', external_var)

iterator = iter([0,1,2,3])


In [56]:
buggy_consume_next(iterator)
# 后面没有正常迭代，输出的都是第一个
buggy_consume_next(iterator)
buggy_consume_next(iterator)

external_var: 0
external_var: 0
external_var: 0


In [57]:
# 按顺序自动执行
a = tf.Variable(1.0)
b = tf.Variable(2.0)

@tf.function
def f(x, y):
    a.assign(y * b)
    b.assign_add(x * a)
    return a + b

f(1.0, 2.0)

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>